In [1]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import csv
import sys
sys.path.append("..")

from Code.VQLSSVM import VQLSSVM
from Code.Utils import prepareDataset

np.set_printoptions(precision=4, suppress=True)

In [13]:
qubits = 4
shots: int = 10000
gamma: float = 0.01 # regularization parameter
subsetSize: int = 2**qubits - 1 # number of training points
classToFilterOut: int = 2

In [14]:
xTrain, xTest, yTrain, yTest = prepareDataset(normalizeValues=True, dataset="breastCancer",subsetSize=subsetSize, classToFilterOut=classToFilterOut)
print(xTrain)

[[-0.1302 -0.5698 -0.1361 -0.4528 -0.1664 -0.2367 -0.6883 -0.4635 -0.0606
  -0.3361 -0.9544 -0.9431 -0.9442 -0.9697 -0.8098 -0.7616 -0.9015 -0.5469
  -0.6775 -0.8064 -0.3568 -0.6205 -0.3756 -0.6676 -0.3089 -0.5581 -0.6623
  -0.1402 -0.3739 -0.5466]
 [-0.1785 -0.0876 -0.0822 -0.4655  0.0521  0.7914  0.4883  0.3688  0.4495
   0.3092 -0.145  -0.0745 -0.1177 -0.508  -0.2761 -0.1333 -0.5842  0.1457
  -0.5641 -0.6443 -0.1519  0.0458 -0.0708 -0.4884  0.0464 -0.0494 -0.0126
   0.6921 -0.3251 -0.3853]
 [ 0.0127 -0.254   0.0173 -0.3042  0.0629 -0.0975 -0.1307  0.0477 -0.0798
  -0.5072 -0.4932 -0.5404 -0.5493 -0.6746 -0.5021 -0.2897 -0.7371 -0.3025
  -0.7214 -0.7186 -0.1078 -0.3022 -0.1783 -0.451  -0.0672 -0.3742 -0.4276
   0.0412 -0.646  -0.7069]
 [ 0.9158 -0.1775  0.9115  0.7871  0.025   0.0553  0.3454  0.8668 -0.2525
  -0.674  -0.4752 -0.5046 -0.5229 -0.5446 -0.8016 -0.6527 -0.843  -0.5298
  -0.7786 -0.872   0.7922  0.1103  0.6972  0.4898  0.0042 -0.2702 -0.147
   0.8474 -0.491  -0.6618]
 [ 0.

# VQLS-LSSVM state vectors

In [18]:
vqlssvmVectors: VQLSSVM = VQLSSVM(gamma, shots)
vqlssvmVectors.train(xTrain, yTrain, quantumSimulation=False, verbose=True, iterations = 200, method="COBYLA")
vqlssvmVectors.plotCost()

Condition number of the matrix:  3708.0831548820574
LS-SVM Matrix:
 [[  0.       1.       1.     ...   1.       1.       1.    ]
 [  1.     110.9463   2.5824 ...   9.9711  11.8826  11.0473]
 [  1.       2.5824 104.0593 ...   2.6184   2.1675   2.3937]
 ...
 [  1.       9.9711   2.6184 ... 109.5775  11.3761  10.4822]
 [  1.      11.8826   2.1675 ...  11.3761 114.5982  13.3511]
 [  1.      11.0473   2.3937 ...  10.4822  13.3511 112.7153]]
['IIIII', 'IIIIX', 'IIIIZ', 'IIIXI', 'IIIXX', 'IIIXZ', 'IIIYY',
 'IIIZI', 'IIIZX', 'IIIZZ', 'IIXII', 'IIXIX', 'IIXIZ', 'IIXXI',
 'IIXXX', 'IIXXZ', 'IIXYY', 'IIXZI', 'IIXZX', 'IIXZZ', 'IIYIY',
 'IIYXY', 'IIYYI', 'IIYYX', 'IIYYZ', 'IIYZY', 'IIZII', 'IIZIX',
 'IIZIZ', 'IIZXI', 'IIZXX', 'IIZXZ', 'IIZYY', 'IIZZI', 'IIZZX',
 'IIZZZ', 'IXIII', 'IXIIX', 'IXIIZ', 'IXIXI', 'IXIXX', 'IXIXZ',
 'IXIYY', 'IXIZI', 'IXIZX', 'IXIZZ', 'IXXII', 'IXXIX', 'IXXIZ',
 'IXXXI', 'IXXXX', 'IXXXZ', 'IXXYY', 'IXXZI', 'IXXZX', 'IXXZZ',
 'IXYIY', 'IXYXY', 'IXYYI', 'IXYYX', 'IXYYZ', 'I

KeyboardInterrupt: 

In [ ]:
print(vqlssvmVectors.predict(xTest))

[-0.0171+0.j -0.0476+0.j -0.0528+0.j -0.0887+0.j -0.0366+0.j  0.0013+0.j
 -0.0498+0.j -0.1357+0.j -0.112 +0.j -0.0448+0.j -0.1065+0.j -0.1248+0.j
 -0.1203+0.j -0.0023+0.j -0.1159+0.j  0.004 +0.j -0.0966+0.j -0.0668+0.j
 -0.1279+0.j -0.0126+0.j -0.0659+0.j -0.0336+0.j -0.0976+0.j -0.0671+0.j
 -0.0414+0.j -0.04  +0.j -0.1513+0.j -0.0127+0.j -0.1542+0.j -0.1235+0.j
 -0.093 +0.j -0.0608+0.j -0.0995+0.j -0.0958+0.j -0.1501+0.j -0.1343+0.j
 -0.0716+0.j -0.1017+0.j -0.1332+0.j -0.0027+0.j -0.0975+0.j -0.0322+0.j
 -0.1298+0.j -0.0408+0.j -0.0619+0.j -0.0572+0.j -0.0566+0.j -0.1375+0.j
  0.0028+0.j -0.0458+0.j -0.0762+0.j -0.1008+0.j -0.0667+0.j -0.0202+0.j
 -0.0736+0.j -0.0856+0.j -0.0925+0.j -0.0739+0.j -0.0882+0.j -0.05  +0.j
 -0.073 +0.j -0.0793+0.j -0.045 +0.j -0.1142+0.j -0.1092+0.j -0.0659+0.j
 -0.031 +0.j -0.0863+0.j -0.0941+0.j -0.1409+0.j -0.0362+0.j -0.0292+0.j
 -0.1137+0.j -0.0204+0.j -0.1292+0.j  0.0768+0.j -0.0128+0.j  0.0325+0.j
 -0.1362+0.j -0.0361+0.j -0.0749+0.j -0.0567+0.j  0

In [ ]:
print("Accuracy:",vqlssvmVectors.accuracy(xTest, yTest))

Accuracy: 0.4187725631768953


# VQLS-LSSVM quantum

In [ ]:
vqlssvmQuantum: VQLSSVM = VQLSSVM(gamma, shots)
vqlssvmQuantum.train(xTrain, yTrain, quantumSimulation=True, verbose=True, iterations = 200)

vqlssvmQuantum.plotCost()

Condition number of the matrix:  3011.3852421141232
LS-SVM Matrix:
 [[  0.       1.       1.       1.       1.       1.       1.       1.
    1.       1.       1.       1.       1.       1.       1.       1.    ]
 [  1.     105.2993   3.8203   4.6593   4.5084   3.816    3.5294   5.1439
    5.0955   4.3462   4.9269   4.4901   4.1531   4.6718   4.5894   4.7327]
 [  1.       3.8203 112.5224   5.6841  13.7819   9.8722   8.6937   6.5622
    4.3327  10.5649  11.9136   8.9747  11.9245   4.6386   9.9499   7.5501]
 [  1.       4.6593   5.6841 105.5041   6.8843   5.7888   4.6382   6.8713
    4.1727   6.0743   6.8313   5.1905   6.277    4.9419   6.8881   4.4222]
 [  1.       4.5084  13.7819   6.8843 117.1645  13.8481  10.0249   8.2388
    4.7811  13.0576  14.9882  10.9682  15.0276   6.1702  12.6203   8.2796]
 [  1.       3.816    9.8722   5.7888  13.8481 113.995    7.4294   6.1172
    4.7653  10.7698  11.7591   9.6083  12.1456   5.812   10.9242   6.5814]
 [  1.       3.5294   8.6937   4.6382  10.

capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.
Traceback (most recent call last):
  File "/Users/pijus/.pyenv/versions/3.11.4/lib/python3.11/site-packages/scipy/optimize/_cobyla_py.py", line 281, in calcfc
Fatal Python error: F2PySwapThreadLocalCallbackPtr: F2PySwapThreadLocalCallbackPtr: PyLong_AsVoidPtr failed
Python runtime state: initialized
    f = sf.fun(x)
        ^^^^^^^^^
  File "/Users/pijus/.pyenv/versions/3.11.4/lib/python3.11/site-packages/scipy/optimize/_differentiable_functions.py", line 267, in fun
    self._update_fun()
  File "/Users/pijus/.pyenv/versions/3.11.4/lib/python3.11/site-packages/scipy/optimize/_differentiable_functions.py", line 251, in _update_fun
    self._update_fun_impl()
  File "/Users/pijus/.pyenv/versions/3.11.4/lib/python3.11/site-packages/scipy/optimize/_differentiable_functions.py", line 155, in update_fun
    self.f = fun_wrapped(self.x)
             ^^^^^^^^^^^^^^^^^^^
  File "/Users/pijus/.pyenv/versions/3.11.4/lib/

: 

: 

In [ ]:
costCOBYLA = vqlssvmQuantum.getCostHistory()

with open('../SimulationResults/costCOBYLA.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Iteration', 'CostFunction'])
    for i in range(len(costCOBYLA)):
        writer.writerow([i, costCOBYLA[i]])


In [ ]:
print("Predictions:\n", vqlssvmQuantum.predict(xTest))
print("Accuracy: ", vqlssvmQuantum.accuracy(xTest, yTest))

# LSSVM (work in progresss)

In [ ]:
# lssvmMatrix1: np.ndarray = lssvmMatrix(xTrain, gamma, "linearKernel")
# yVector: np.array = prepareLabels(yTrain)
# print(yVector)
# print ("Condition number of the matrix: ", np.linalg.cond(lssvmMatrix1))
# print("LS-SVM Matrix:\n", lssvmMatrix1)
# print(yVector)

# solution: np.array = np.linalg.solve(lssvmMatrix1, yVector)
# print("Solution:", solution)
# print ("Norm of the solution: ",solution/np.linalg.norm(solution))

# bias: float = solution[0]
# weights: np.array = solution[1:]

# vqlssvmLSSVM: VQLSSVM = VQLSSVM(gamma,10)
# vqlssvmLSSVM.xTrain = xTrain
# vqlssvmLSSVM.xTrainSize = xTrain.shape[0]
# vqlssvmLSSVM.weights = weights
# vqlssvmLSSVM.b = bias
# predictions: np.array = vqlssvmLSSVM.predict(xTest)
# classVals = [vqlssvmLSSVM.assignClass(i) for i in predictions]
# print("Predictions:", predictions)
# print("Class values:", classVals)
# print("yTest:", yTest)
# # print(yTest)
# print("Accuracy:", vqlssvmLSSVM.accuracy(xTest, yTest))

# Classical SVM

In [17]:
clf = SVC(kernel='linear')
clf.fit(xTrain, yTrain)
yPred = clf.predict(xTest)
print("Accuracy:", accuracy_score(yTest, yPred))

Accuracy: 0.9479553903345725
